## Anexo: ¿Qué hacemos con las referencias circulares?

Basado en la respuesta de [@indonoso](https://github.com/IIC2233-2016-1/syllabus/issues/481#issuecomment-221145469) a una pregunta sobre este tema.

Antes de discutir qué hacer con las referencias circulares, definamos a qué nos estamos refiriendo.

Entenderemos como **referencias circulares** cuando dos clases tienen una relación de composición/agregación mutua. Es decir, una clase contiene a la otra y viceversa.

En el caso típico de modelos circulares tenemos una clase que _organiza o es dueña_ de los otros objetos, y estos objetos necesitan conocer el estado de la organización para poder realizar acciones.

La solución típica (y errónea) es la siguiente:

In [1]:
class Organizador:
    def __init__(self, num_elementos):
        self.listo = True
        self.elementos = [Elemento(self) for _ in range(num_elementos)]

class Elemento:
    def __init__(self, organizador):
        self.organizador = organizador

    def realizar_accion(self):
        if self.organizador.listo:
            print("Me puedo mover!!! :D")
        else:
            print("Mi jefe dice que no me puedo mover :'(")

En el código anterior, `Organizador` contiene instancias de `Elemento`, pero a la vez, cada `Elemento` contiene la instancia de su `Organizador`. La modelación es circular porque un `Elemento` puede acceder a todos los atributos de `Organizador` y modificarlos si quisiera. En la vida real esta situación casi no ocurre, por ejemplo:

- Los alumnos pueden hacer preguntas en el _syllabus_ de GitHub, pero no pueden modificar las actividades.
- Los clientes pueden saber cuántos productos hay en vitrina de un supermercado, pero no conocen sus facturas, ni su sistema de logística

Por esto es que esta modelación **no se aproxima a la realidad**. Distintos lenguajes de programación ofrecen herramientas para lograr una modelación más precisa (Por ejemplo, C# tiene las interfaces en conjunto con modificadores de acceso [[1]](https://blogs.msdn.microsoft.com/nickmalik/2005/03/18/how-to-get-rid-of-circular-references-in-c/)).

Para modelar esto en Python se usa el hecho de que las funciones son objetos, y que por lo tanto, podemos definirlas dentro de otras funciones, pasarlas como parámetro o retornarlas. El siguiente código es una forma de arreglar el código de arriba.

In [2]:
class Organizador:
    def __init__(self, num_elementos):
        self.listo = True
        self.elementos = [Elemento(self.esta_listo) for _ in range(num_elementos)]

    def esta_listo(self):
        return self.listo

class Elemento:
    def __init__(self, puedo_realizar_accion):
        self.puedo_realizar_accion = puedo_realizar_accion

    def realizar_accion(self):
        if self.puedo_realizar_accion():
            print("Me puedo mover!!! :D")
        else:
            print("Mi jefe dice que no me puedo mover :'(")

Aquí pasamos **el método** `self.esta_listo`  que retorna el valor del atributo listo de `Organizador`. Luego `Elemento` usa esta función misma función guardada en la variable `puedo_realizar_accion` para "hacerle una pregunta" a `Organizador`.

Hemos aprendido que la forma de evitar estas dependencias circulares es pasando funciones en vez de la instancia completa. Sin embargo, cuando el método a entregar tenga parámetros que sólo la clase que lo define debería _setear_, necesitaremos **construir** un método con esos parámetros ya configurados. Por ejemplo, si `esta_listo` dependiera de un secreto que sólo el `Organizador` conoce, lo haríamos de esta manera:

In [3]:
class Organizador:
    def __init__(self, num_elementos):
        self.listo = True
        self.elementos = [Elemento(self.esta_listo(i)) for i in range(num_elementos)]

    def esta_listo(self, secreto):
        def _esta_listo():
            # Devuelve True si está listo y el elemento que pregunta
            # Está ubicado en una posición impar de la lista (secreto... shh!!)
            return self.listo and secreto % 2
        return _esta_listo

class Elemento:
    def __init__(self, puedo_realizar_accion):
        self.puedo_realizar_accion = puedo_realizar_accion

    def realizar_accion(self):
        if self.puedo_realizar_accion():
            print("Me puedo mover!!! :D")
        else:
            print("Mi jefe dice que no me puedo mover :'(")